# Description

This notebook can be used to test the implementation of the elastic energy and the elastic forces.

# Load libraries

In [ ]:
import torch
import igl
import meshplot as mp
import time

import sys as _sys
_sys.path.append("../src")
from fem_system import *
from elastic_energy import *
from utils import *
from matplotlib import gridspec
import matplotlib.pyplot as plt

shadingOptions = {
    "flat":True,
    "wireframe":False,   
}

rot = torch.tensor(
    [[1.0,  0.0, 0.0],
     [0.0,  0.0, 1.0],
     [0.0, -1.0, 0.0]]
)

# Load mesh

Several meshes are available for you to play with under `data/`: `ball.obj`, `dinosaur.obj`, and `beam.obj`.

In [ ]:
v, _, _, t, _, _ = igl.read_obj("../data/beam.obj")
v = torch.tensor(v)
t = torch.tensor(t)

aabb = torch.max(v, dim=0).values - torch.min(v, dim=0).values
length_scale = torch.mean(aabb)

# Linear/Non-Linear Elastic Solid

## Instantiation

We first specify the elasticity model to use for the elastic solid, as well as pinned vertices, and volumetric forces.

In [ ]:
rho     = 131.0 # [kg.m-3]
young   = 1.0e7 # [Pa] 
poisson = 0.2
force_mass = torch.zeros(size=(3,))
force_mass[2] = - rho * 9.81

minX    = torch.min(v[:, 0])
pin_idx = list(torch.arange(v.shape[0])[v[:, 0] < minX + 0.2*aabb[0]])

ee    = LinearElasticEnergy(young, poisson)
# ee    = NeoHookeanElasticEnergy(young, poisson)
solid = FEMSystem(v, t, ee, rho=rho, pin_idx=pin_idx, f_mass=force_mass)


# Validate Elastic Energy Gradient (Elastic Forces) using Finite Differences on Elastic Energy

In [ ]:
fd_validation_elastic(solid, v.clone())

# Validate External Energy Gradient (External Forces) using Finite Differences on External Energy

In [ ]:
fd_validation_ext(solid, v.clone())